In [13]:
import argparse
import time
from collections import defaultdict
from pathlib import Path
import h5py
import fastmri
import fastmri.data.transforms as T
import numpy as np
import requests
import torch
from fastmri.data import SliceDataset
from fastmri.models import VarNet
from tqdm import tqdm
from fastmri.data.subsample import create_mask_for_mask_type

In [14]:
# loading  multi coil knee file 

fname = '/scratch/svangurp/samuel/data/knee/train/file1000002.h5'
data = h5py.File(fname, 'r')
kspace = data["kspace"][()]


In [59]:
#  masking of the multi coil knee kspace 
slice = 10 # picking a random slice to use 

# create random mask
mask_func = create_mask_for_mask_type(mask_type_str="random", center_fractions=[0.08], accelerations=[4])

# transforming one slice of the multi coil k-space data
_kspace = T.to_tensor(kspace)[slice]

# applying the mask to the one slice of the multi coil k-space data
masked_kspace, mask = T.apply_mask(_kspace, mask_func)


In [29]:
def run_varnet_model(batch, model, device):
    masked_kspace, mask, _, fname, slice_num, _, crop_size = batch
    crop_size = crop_size[0]  # always have a batch size of 1 for varnet

    output = model(masked_kspace.to(device), mask.to(device)).cpu()

    # detect FLAIR 203
    if output.shape[-1] < crop_size[1]:
        crop_size = (output.shape[-1], output.shape[-1])

    output = T.center_crop(output, crop_size)[0]

    return output, int(slice_num[0]), fname[0]

In [30]:
def run_inference(challenge, state_dict_file, data_path, output_path, device):
    model = VarNet(num_cascades=12, pools=4, chans=18, sens_pools=4, sens_chans=8)
    # download the state_dict if we don't have it
    if state_dict_file is None:
        if not Path(MODEL_FNAMES[challenge]).exists():
            url_root = VARNET_FOLDER
            download_model(url_root + MODEL_FNAMES[challenge], MODEL_FNAMES[challenge])

        state_dict_file = MODEL_FNAMES[challenge]

    model.load_state_dict(torch.load(state_dict_file))
    model = model.eval()

    # data loader setup
    data_transform = T.VarNetDataTransform()
    dataset = SliceDataset(
        root=data_path, transform=data_transform, challenge="multicoil"
    )
    dataloader = torch.utils.data.DataLoader(dataset, num_workers=4)

    # run the model
    start_time = time.perf_counter()
    outputs = defaultdict(list)
    model = model.to(device)

    for batch in tqdm(dataloader, desc="Running inference"):
        with torch.no_grad():
            output, slice_num, fname = run_varnet_model(batch, model, device)

        outputs[fname].append((slice_num, output))

    # save outputs
    for fname in outputs:
        outputs[fname] = np.stack([out for _, out in sorted(outputs[fname])])

    fastmri.save_reconstructions(outputs, output_path / "reconstructions")

    end_time = time.perf_counter()

    print(f"Elapsed time for {len(dataloader)} slices: {end_time-start_time}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    parser.add_argument(
        "--challenge",
        default="varnet_knee_mc",
        choices=(
            "varnet_knee_mc",
            "varnet_brain_mc",
        ),
        type=str,
        help="Model to run",
    )
    parser.add_argument(
        "--device",
        default="cuda",
        type=str,
        help="Model to run",
    )
    parser.add_argument(
        "--state_dict_file",
        default=None,
        type=Path,
        help="Path to saved state_dict (will download if not provided)",
    )
    parser.add_argument(
        "--data_path",
        type=Path,
        required=True,
        help="Path to subsampled data",
    )
    parser.add_argument(
        "--output_path",
        type=Path,
        required=True,
        help="Path for saving reconstructions",
    )

    args = parser.parse_args()

    run_inference(
        args.challenge,
        args.state_dict_file,
        args.data_path,
        args.output_path,
        torch.device(args.device),
    )



usage: ipykernel_launcher.py [-h]
                             [--challenge {varnet_knee_mc,varnet_brain_mc}]
                             [--device DEVICE]
                             [--state_dict_file STATE_DICT_FILE] --data_path
                             DATA_PATH --output_path OUTPUT_PATH
ipykernel_launcher.py: error: the following arguments are required: --data_path, --output_path


SystemExit: 2

/scratch/svangurp/envs/fastMRI/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [51]:
model  = '/scratch/svangurp/samuel/pretrained/knee/varnet/knee_leaderboard_state_dict.pt'

crop_size = (320,320)
slice_num = 1 # ??
batch = masked_kspace, mask, _, fname, slice_num, _, crop_size 
device = torch.device('cuda')   
output = run_varnet_model(batch, model, device)



TypeError: 'str' object is not callable

In [67]:
challenge = 'varnet_knee_mc' 
state_dict_file ='/scratch/svangurp/samuel/pretrained/knee/varnet/knee_leaderboard_state_dict.pt'
data_path = '/scratch/svangurp/samuel/data/knee/train/'
output_path ='/home/svangurp/scratch/samuel/data/knee/varnet_recon_knee/'
device = 'cuda' 

run_inference(challenge, state_dict_file, data_path, output_path, device)

Running inference:   0%|          | 0/1448 [00:00<?, ?it/s]


AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/scratch/svangurp/envs/fastMRI/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/scratch/svangurp/envs/fastMRI/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/scratch/svangurp/envs/fastMRI/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/svangurp/PycharmProjects/fastMRI/fastmri/data/mri_data.py", line 358, in __getitem__
    sample = self.transform(kspace, mask, target, attrs, fname.name, dataslice)
  File "/home/svangurp/PycharmProjects/fastMRI/fastmri/data/transforms.py", line 399, in __call__
    mask = torch.from_numpy(mask.reshape(*mask_shape).astype(np.float32))